## Dynamic Programing

##### 1. Rod Cutting

In [ ]:
def cut_rod(p,n):
    if n == 0:
        return 0
    q = -1
    for i in range(1,n+1):
        q = max(q, p[i] + cut_rod(p,n-i))
    return q

p = {1:1, 2:5, 3:8, 4:9, 5:10, 6:17, 7:17, 8:20, 9:24, 10:30, 11:32, 12:34, 13:38, 14:40, 15:43, 16:47}
# cut_rod(p,2)

import timeit
# %timeit cut_rod(p,16)

def memoized_cut_rod(p,n):
    r = {}
    for i in range(0,n+1):
        r[i] = -1
    return memoized_cut_rod_aux(p,n,r)

def memoized_cut_rod_aux(p,n,r):
    if r[n] >= 0: # O(1) lookup from dicts' key
        return r[n]
    if n == 0:
        q = 0
    else:
        q = -1
        for i in range(1,n+1):
            q = max(q, p[i] + memoized_cut_rod_aux(p,n-i,r))

    r[n] = q
    return q



# %timeit memoized_cut_rod(p,16)
# %timeit cut_rod(p,16)

def bottom_up_cut_rod_ext(p,n):
    r = {}
    s = {}
    r[0] = 0
    for j in range(1,n+1):
        q = -1
        for i in range(1,j+1):
            if q < p[i] + r[j-i]:
                q = max(q, p[i] + r[j-i]) # requires r[1] ==> r[2-1] for 2nd loop; first loop r[j-i=0] = 0. i.e. j = i =1
                s[j] = i
        r[j] = q # record the computed result of the optimal subproblem solution
    return r, s

def print_bottom_up_cut_rod(p,n):
    r, s = bottom_up_cut_rod_ext(p,n)
    while n > 0:
        print(s[n])
        n = n - s[n]

# %timeit cut_rod(p,16)
# %timeit bottom_up_cut_rod(p,16)
# %timeit memoized_cut_rod(p,16)
n = 4
bottom_up_cut_rod_ext(p,n), print_bottom_up_cut_rod(p,n)

In [ ]:
print(p)

##### 2. Matrix Multiplication

###### 2.1 Bottom up DP

In [ ]:
import numpy as np
p1 = {0: 30, 1:35, 2:15, 3:5, 4:10, 5:20, 6:25}
p2 = {0: 30, 1:35, 2:15, 3:5, 4:10, 5:20, 6:25, 7: 40, 8: 80}

def matrix_multi_parens(p):
    n = len(p)-1
    m = {}
    s = {}

    for i in range(1,n+1):
        m[(i,i)] = 0

    for l in range(2,n+1):
        for i in range(1,n-l+2):
            j = i+l-1
            m[(i,j)] = np.inf
            for k in range(i,j):
                q = m[(i,k)] + m[(k+1,j)] + p[i-1]*p[k]*p[j]
                if q < m[(i,j)]:
                    m[(i,j)] = q
                    s[(i,j)] = k
    return m,s

m1,s1 = matrix_multi_parens(p1)
m2,s2 = matrix_multi_parens(p2)
m1

In [ ]:
def print_optimal_parens(s,i,j):
    if i == j:
        print(f"A{i}", end="")
    else:
        print("(", end="")
        print_optimal_parens(s,i,s[(i,j)])
        print_optimal_parens(s,s[(i,j)]+1,j)
        print(")",end="")

result = print_optimal_parens(s1,1,6)
result2 = print_optimal_parens(s2,1,8)
result, result2

###### 2.2 Top down recursive

In [ ]:
import numpy as np
def recursive_matrix_chain(p,i,j):
    if i == j:
        return 0
    m = {}
    m[(i,j)] = np.inf
    for k in range(i,j):
        q = recursive_matrix_chain(p,i,k) + recursive_matrix_chain(p,k+1,j) + p[i-1]*p[k]*p[j]
        if q < m[(i,j)]:
            m[(i,j)] = q
    return m[(i,j)]

p = {0: 30, 1:35, 2:15, 3:5, 4:10, 5:20, 6:25}
recursive_matrix_chain(p,1,6)

###### 2.3 Memoized top-down DP

In [ ]:
def memoized_matrix_chain(p):
    n = len(p) - 1
    m = {}
    for i in range(1,n+1):
        for j in range(1,n+1):
            m[(i,j)] = np.inf
    return lookup_chain(m,p,1,n)

def lookup_chain(m,p,i,j):
    if m[(i,j)] < np.inf:
        return m[(i,j)]
    if i == j:
        m[(i,j)] = 0
    else:
        for k in range(i,j):
            q = lookup_chain(m,p,i,k) + lookup_chain(m,p,k+1,j) + p[i-1]*p[k]*p[j]
            if q < m[(i,j)]:
                m[(i,j)] = q
    return m[(i,j)]

memoized_matrix_chain(p)

##### 3. Long Common Subsequence (LCS)

In [ ]:
def LCS(X,Y):
    m = len(X)
    n = len(Y)
    b = {} # store the arrow
    c = {} # store the computed LCS
    c[(0,0)] = 0
    for i in range(1,m+1):
        c[(i,0)] = 0
    for j in range(1,n+1):
        c[(0,j)] = 0
    for i in range(1,m+1):
        for j in range(1,n+1):
            if X[i] == Y[j]:
                c[(i,j)] = c[(i-1,j-1)] + 1
                b[(i,j)] = "&"
            elif c[(i-1,j)] >= c[(i,j-1)]:
                c[(i,j)] = c[(i-1,j)]
                b[i,j] = "^"
            else:
                c[(i,j)] = c[i,j-1]
                b[(i,j)] = "<"
    return c, b

X = {1:'a',2:'b',3:'c',4:'b',5:'d',6:'a',7:'b'}
Y = {1:'b',2:'d',3:'c',4:'a',5:'b',6:'a'}

c,b = LCS(X,Y)

def print_LCS(b,X,i,j):
    if i == 0 or j == 0:
        return
    if b[(i,j)] == "&":
        print_LCS(b,X,i-1,j-1)
        print(X[i],end="")
    elif b[(i,j)] == "^":
        print_LCS(b,X,i-1,j)
    else:
        print_LCS(b,X,i,j-1)

print_LCS(b,X,7,6)

##### 4. Optimal Binary Search Tree (BST)

In [ ]:
def optimal_BST(p,q,n):
    e = {} # expected value
    w = {}
    for i in range(1,n+2):
        e[(i,i-1)] = q[i-1]
        w[(i,i-1)] = q[i-1]
    
    for l in range(1,n+1):
        for i in range(1,n-l+2):
            j = i+l-1
            e[(i,j)] = np.inf
            w[(i,j)] = w[(i,j-1)] + p[j] + q[j]
            for r in range(i,j+1):
                t = e[(i,r-1)] + e[(r+1,j)] + w[(i,j)]
                if t < e[(i,j)]:
                    e[(i,j)] = t
                    w[(i,j)] = r
    return w, e


p = {1:0.15, 2:0.10, 3:0.05, 4:0.10, 5:0.20}
q = {0:0.05, 1:0.10, 2:0.05, 3:0.05, 4:0.05, 5:0.10}
n = 5

w, e = optimal_BST(p,q,n)

w, e

# TODO

#### 5. Longest Increasing Subsequence (LIS)

#### 6. Longest Common Palindromic Sequence

#### 7. Alternating Coins Game

#### 8. Subset Sum

#### 9. Edit Distance

#### 10. Knapsack Problem